In [2]:
from llm.LLMModel import *
from ner.llm_ner.prompt_techniques.pt_abstract import PromptTechnique
from ner.llm_ner.prompt_techniques.pt_discussion import PT_OutputList
from ner.llm_ner.prompt_techniques.pt_gpt_ner import PT_GPT_NER
from ner.llm_ner.prompt_techniques.pt_gpt_ner import PT_GPT_NER
from ner.llm_ner.prompt_techniques.pt_wrapper import PT_Wrapper
from ner.llm_ner.few_shots_techniques import *
from ner.llm_ner.ResultInstance import load_all_results, load_result
import matplotlib.pyplot as plt
%reload_ext autoreload
%autoreload 2

In [65]:
df_results = load_all_results(root_directory = "ner/saves/results/ontonote5/mistral-7b-v0.1/")
df_results

/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value enco

,f1_mean,f1_conf_inter,model,noshots,prompt_technique,few_shot_tecnique,nb_few_shots,precision,plus_plus,verifier,len_data_train,len_data_test,nb_test_run,confidence_interval,distribution_used,precision_mean,precision_conf_inter,recall_mean,recall_conf_inter
4,0.923,"(0.877, 0.969)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,False,discussion,entity,5,False,False,None,1538,50,3,0.95,Student,0.919,"(0.831, 1.007)",0.873,"(0.800, 0.946)"
5,0.912,"(0.836, 0.988)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,False,discussion,sentence,5,True,False,None,1538,50,3,0.95,Student,0.924,"(0.817, 1.031)",0.855,"(0.740, 0.970)"
3,0.890,"(0.845, 0.936)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,False,discussion,random,5,True,False,None,1538,50,3,0.95,Student,0.870,"(0.813, 0.927)",0.823,"(0.750, 0.895)"
10,0.890,"(0.846, 0.935)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,False,discussion,entity,5,True,False,None,1538,50,3,0.95,Student,0.875,"(0.744, 1.006)",0.821,"(0.757, 0.885)"
6,0.888,"(0.840, 0.935)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,False,discussion,sentence,5,False,False,None,1538,50,3,0.95,Student,0.875,"(0.839, 0.911)",0.827,"(0.772, 0.883)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,0.000,"(0.000, 0.000)",mistral-7b-v0.1-ft-wrapper-2000-Q5_0,False,wrapper,no-shots,5,True,False,None,1538,50,10,0.95,Student,0.000,"(0.000, 0.000)",0.000,"(0.000, 0.000)"
88,0.000,"(0.000, 0.000)",mistral-7b-v0.1,False,multi_prompt-get-entities-tagger,no-shots,0,300,False,None,1303,100,3,0.95,Student,0.000,"(0.000, 0.000)",0.000,"(0.000, 0.000)"
86,0.000,"(0.000, 0.000)",mistral-7b-v0.1,False,multi_prompt-get-entities-tagger,no-shots,0,300,True,None,1303,100,3,0.95,Student,0.000,"(0.000, 0.000)",0.000,"(0.000, 0.000)"
43,0.000,"(0.000, 0.000)",mistral-7b-v0.1-ft-wrapper-2000-Q5_0,False,wrapper,no-shots,5,False,False,None,1538,50,10,0.95,Student,0.000,"(0.000, 0.000)",0.000,"(0.000, 0.000)"


In [67]:
df_to_show = df_results[~df_results['model'].str.contains('ft')][['f1_mean', 'f1_conf_inter', 'prompt_technique',
       'few_shot_tecnique', 'nb_few_shots', 'precision', 'plus_plus']]

df_res = df_to_show[df_to_show['precision'] == '300']
df_res['tech_name'] = df_res.apply(lambda row :f"With {row['nb_few_shots']} few_shots {'and ++' if row['plus_plus']  else ''}", axis = 1)
df_res

/tmp/ipykernel_8826/2272134630.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res['tech_name'] = df_res.apply(lambda row :f"With {row['nb_few_shots']} few_shots {'and ++' if row['plus_plus']  else ''}", axis = 1)


,f1_mean,f1_conf_inter,prompt_technique,few_shot_tecnique,nb_few_shots,precision,plus_plus,tech_name
96,0.323,"(0.297, 0.350)",wrapper,sentence,3,300,True,With 3 few_shots and ++
95,0.321,"(0.275, 0.367)",wrapper,sentence,3,300,False,With 3 few_shots
90,0.246,"(0.216, 0.275)",multi_prompt-get-entities-tagger,sentence,3,300,True,With 3 few_shots and ++
85,0.238,"(0.202, 0.274)",discussion,sentence,3,300,True,With 3 few_shots and ++
89,0.238,"(0.205, 0.271)",multi_prompt-get-entities-tagger,sentence,3,300,False,With 3 few_shots
84,0.236,"(0.194, 0.277)",discussion,sentence,3,300,False,With 3 few_shots
80,0.161,"(0.132, 0.191)",filing,sentence,3,300,True,With 3 few_shots and ++
79,0.152,"(0.098, 0.205)",filing,sentence,3,300,False,With 3 few_shots
81,0.119,"(0.103, 0.135)",discussion,no-shots,0,300,True,With 0 few_shots and ++
83,0.110,"(0.100, 0.119)",discussion,no-shots,0,300,False,With 0 few_shots


In [78]:
from ner.plot_results import show_diff_plus_plus


results, df_results = show_diff_plus_plus()
print(results)
df_results

/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value enco

The mean difference between plus plus and raw is 0.004 with confidence 95% student interval (0.003, 0.005)


plus_plus,prompt_technique,few_shot_tecnique,nb_few_shots,precision,result,True,False
0,discussion,no-shots,0,300,0.009,0.119,0.110
1,discussion,sentence,3,300,0.002,0.238,0.236
2,filing,no-shots,0,300,0.007,0.088,0.081
3,filing,sentence,3,300,0.009,0.161,0.152
4,multi_prompt-get-entities-tagger,no-shots,0,300,0.000,0.000,0.000
5,multi_prompt-get-entities-tagger,sentence,3,300,0.008,0.246,0.238
6,wrapper,no-shots,0,300,-0.005,0.021,0.026
7,wrapper,sentence,3,300,0.002,0.323,0.321


In [85]:
from ner.plot_results import show_diff_few_shots


results, df_results = show_diff_few_shots()
print(results)
df_results

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/root/miniconda3/lib/python3.11/sit

The mean difference between plus plus and raw is 0.184 with confidence 95% student interval (0.155, 0.212)


/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/root/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value enco

nb_few_shots,prompt_technique,plus_plus,precision,result,3,0
0,discussion,False,300,0.126,0.236,0.110
1,discussion,True,300,0.119,0.238,0.119
2,filing,False,300,0.071,0.152,0.081
3,filing,True,300,0.073,0.161,0.088
4,multi_prompt-get-entities-tagger,False,300,0.238,0.238,0.000
5,multi_prompt-get-entities-tagger,True,300,0.246,0.246,0.000
6,wrapper,False,300,0.295,0.321,0.026
7,wrapper,True,300,0.302,0.323,0.021


## Show results for finetuned models

In [ ]:

df_results['finetuned'] = df_results['model'].map(lambda s : 'ft' in s)
df_results['nb_samples_tested'] = df_results['nb_test_run'] * df_results['len_data_test']
ft_results = df_results[df_results['finetuned']]
ft_results = ft_results[['f1_mean', 'f1_conf_inter', 'prompt_technique', 'model', 'noshots', 'few_shot_tecnique','nb_few_shots', 'nb_samples_tested']]
ft_results['model_base'] = ft_results['model'].map(lambda s : '-'.join(s.split('-')[:-4]))
ft_results['nb_train_samples'] = ft_results['model'].map(lambda s : s.split('-')[-2])
ft_results = ft_results.sort_values(['prompt_technique',	'few_shot_tecnique', 'f1_mean'], ascending=False)

In [ ]:
# ['f1_mean', 'f1_conf_inter', 'prompt_technique', 'few_shot_tecnique',
    #    'nb_few_shots', 'nb_samples_tested', 'model_base', 'nb_train_samples',
    #    'finetuned']
grouped = ft_results.groupby(['prompt_technique', 'few_shot_tecnique', 'noshots', 
       'nb_few_shots', 'nb_samples_tested', 'model_base', 'nb_train_samples'])
grouped 

idx_max = grouped['f1_mean'].idxmax()

ft_cleaned = ft_results.loc[idx_max].sort_values('nb_train_samples', ascending=False)

df = ft_cleaned#[ft_cleaned['nb_samples_tested'] == 500]
# Create dummy columns for 'few_shot_tecnique'
df.pivot(index=['model_base','prompt_technique','noshots',
        'nb_train_samples','nb_samples_tested'], columns='few_shot_tecnique', values= 'f1_mean')

## Show results for non finetuned models

In [ ]:
df_results['finetuned'] = df_results['model'].map(lambda s : 'ft' in s)
df_results['nb_samples_tested'] = df_results['nb_test_run'] * df_results['len_data_test']
ft_results = df_results[~df_results['finetuned']]
ft_results = ft_results[['f1_mean', 'f1_conf_inter', 'prompt_technique', 'model', 'noshots', 'few_shot_tecnique','nb_few_shots', 'nb_samples_tested']]
ft_results = ft_results.sort_values(['prompt_technique',	'few_shot_tecnique', 'f1_mean'], ascending=False)

In [ ]:
# ['f1_mean', 'f1_conf_inter', 'prompt_technique', 'few_shot_tecnique',
    #    'nb_few_shots', 'nb_samples_tested', 'model_base', 'nb_train_samples',
    #    'finetuned']
ft_results = ft_results[ft_results['model'] != 'none']
grouped = ft_results.groupby([ 'model', 'prompt_technique', 
       'nb_few_shots', 'nb_samples_tested','few_shot_tecnique', ])
grouped 

# idx_max = grouped['f1_mean'].idxmax()
# idx_max

# Create dummy columns for 'few_shot_tecnique'
ft_results.loc[idx_max].pivot(index=['model','prompt_technique', 'nb_few_shots','nb_samples_tested',
        ], columns='few_shot_tecnique', values= 'f1_mean')

## Showing prompts 

In [ ]:
LLMModel.show_prompts()

## Looking at results of confidence with the tagger

In [ ]:
from ner.utils import load

res = load ("ner/saves/results/conll2003_cleaned/mistral-7b-v0.1/multi_prompt-get-entities-tagger/sentence_3_None_1538_50_1_False.pkl")

cm, f1, precision, recall, y_true, y_pred, y_conf= res.res_insts[0].get_scores(with_y = True)
conf = []
for sent in res.res_insts[0].results:
    conf.extend([s[2] for s in sent])

len(y_conf), len(y_true), len(y_pred)
# [i for i, tag in enumerate(y_pred) if tag == 'None']
right = [y_true[i] == y_pred[i] for i in range(len(y_true))]
rand = [i for i in range(len(right))]

In [ ]:
from collections import Counter

# Example list
my_list = list(zip(y_conf, right))

# Use Counter to count occurrences
entry_counts = Counter(sorted(my_list))

# Print the counts
for entry, count in entry_counts.items():
    print(f'{entry}: {count} times')

In [ ]:
import seaborn as sns
df_conf = pd.DataFrame(np.array([right, y_conf]).T, columns = ['Right', 'Confidence'])

df_conf = pd.DataFrame(df_conf.groupby(['Right', 'Confidence'])['Right'].count()).rename(columns = {'Right' : "count"}).reset_index()
sns.set(rc={'figure.figsize':(10,10)})
order = ['high', 'medium-high', 'medium', 'medium-low', 'low']
sns.catplot(data=df_conf, x="Confidence", y="count", hue="Right", kind="bar",aspect=1.5, order = order)


## Analysing specific result

In [8]:
model  = "mistral-7b-v0.1"
result = load_result(model, PT_OutputList.name(), FST_Sentence.name(), nb_few_shots= 10, with_precision = "10_True")

for r in result.res_insts :
    r.show_cm()
    r.analyse_results()

AttributeError: 'ResultInstance' object has no attribute 'tags'

In [ ]:
r.f1

In [ ]:
from ner.Datasets.Conll2003Dataset import Conll2003Dataset
from ner.Datasets.MyDataset import MyDataset
data = MyDataset.my_load_dataset(dataset=Conll2003Dataset, split = 'test', cleaned= True)
[(d['text'], d['spans']) for d in data if str(d['spans']) == """[['Switzerland', 'LOC'], ["Alfonse D'Amato", 'PER'], ['U.S. Senate Banking Committee', 'ORG'], ['Poland', 'LOC'], ['Polish', 'MISC'], ['Swiss', 'MISC'], ['Poland', 'LOC']]"""]

In [ ]:
[(d['text'], d['spans']) for d in data if 'Hansenne' in d['text']]